In [108]:
import pandas as pd
import mysql.connector
import warnings
warnings.filterwarnings('ignore')

In [110]:
#caminho de destinho 
#path_excel = 'copasa.xlsx'
#path_excel = 'Medição_copasa.xlsx'
#Transformando em dataframe
path_excel = 'COPASA_NOV_24.xlsx'
df_copasa = pd.read_excel(path_excel)

#Exibindo
df_copasa.head(1)

,Localidade,Projeto,Prog. Orçamentário,Requisição,Diagrama de rede,Texto Diagrama de rede,Analista/Fiscal,Planejado,Compromisso,Total Medido,Medido c/reajuste,Saldo,Status,Data Status,Desvio
0,ARAXÁ,E23044-1/PB,PO24.CS.PRE001,12182807,500125117,CT: 22.1544 - YC ENG. - OS.: 4.46/2022,30760,309904.77,309904.77,99261.68,104035.95,210643.09,LIB,2024-01-19,67.97


In [112]:
#Visualizando o nome as colunas
df_copasa.columns

Index(['Localidade', 'Projeto', 'Prog. Orçamentário', 'Requisição',
       'Diagrama de rede', 'Texto Diagrama de rede', 'Analista/Fiscal',
       'Planejado', 'Compromisso', 'Total Medido', 'Medido c/reajuste',
       'Saldo', 'Status', 'Data Status', 'Desvio'],
      dtype='object')

In [116]:
#modificando o nome das colunas
df_copasa.rename(columns={
    'Localidade':'LOCALIDADE',
    'Projeto':'PROJETO',
    'Prog. Orçamentário':'PROG_ORCAMENTARIO',
    'Requisição':'REQUISICAO',
    'Diagrama de rede':'DIAGRAMA_DE_REDE',
    'Texto Diagrama de rede':'TEXTO_DIAGRAMA',
    'Analista/Fiscal':'ANALISTA_FISCAL',
    'Planejado':'PLANEJADO',
    'Compromisso':'COMPROMISSO',
    'Total Medido':'TOTAL_MEDIDO',
    'Medido c/reajuste':'MEDIDO_REAJUSTE',
    'Saldo':'SALDO',
    'Status':'STATUS',
    'Data Status':'DATA_STATUS',
    'Desvio':'DESVIO'}, inplace=True)

In [118]:
df_copasa.columns

Index(['LOCALIDADE', 'PROJETO', 'PROG_ORCAMENTARIO', 'REQUISICAO',
       'DIAGRAMA_DE_REDE', 'TEXTO_DIAGRAMA', 'ANALISTA_FISCAL', 'PLANEJADO',
       'COMPROMISSO', 'TOTAL_MEDIDO', 'MEDIDO_REAJUSTE', 'SALDO', 'STATUS',
       'DATA_STATUS', 'DESVIO'],
      dtype='object')

In [98]:
#Identificando modifcando o tipo
#df_copasa['OBSERVACAO'] = df_copasa['OBSERVACAO'].astype(str)

In [120]:
#/modificando a data para o tipo string
df_copasa['DATA_STATUS'] = df_copasa['DATA_STATUS'].astype(str)

In [100]:
#df_copasa['OBSERVACAO'].fillna('null',inplace=True)

In [122]:
df_copasa.isnull().sum()

LOCALIDADE           0
PROJETO              0
PROG_ORCAMENTARIO    0
REQUISICAO           0
DIAGRAMA_DE_REDE     0
TEXTO_DIAGRAMA       0
ANALISTA_FISCAL      0
PLANEJADO            0
COMPROMISSO          0
TOTAL_MEDIDO         0
MEDIDO_REAJUSTE      0
SALDO                0
STATUS               0
DATA_STATUS          0
DESVIO               0
dtype: int64

df_copasa['PROJETO'] = df_copasa['PROJETO'].fillna('null')

df_copasa['PROG_ORCAMENTARIO'] = df_copasa['PROG_ORCAMENTARIO'].fillna('null')

df_copasa['REQUISICAO'] = df_copasa['REQUISICAO'].fillna('null')

df_copasa['ANALISTA_FISCAL'] = df_copasa['ANALISTA_FISCAL'].fillna('null')

df_copasa['DIAGRAMA_DE_REDE'] = df_copasa['DIAGRAMA_DE_REDE'].fillna('null')

df_copasa['STATUS'] = df_copasa['STATUS'].fillna('null')

#df_copasa['COORDENADORES'] = df_copasa['COORDENADORES'].fillna('null')

df_copasa['MEDIDO_REAJUSTE'] = df_copasa['MEDIDO_REAJUSTE'].fillna(0)

df_copasa['DESVIO'] = df_copasa['DESVIO'].fillna(0)

df_copasa['TEXTO_DIAGRAMA'] = df_copasa['TEXTO_DIAGRAMA'].fillna(0)

df_copasa['LOCALIDADE'] = df_copasa['LOCALIDADE'].fillna(0)

#df_copasa['DATA_STATUS'] = df_copasa['DATA_STATUS'].replace('NaT',00-00-0000)


In [124]:
df_copasa.dtypes

LOCALIDADE            object
PROJETO               object
PROG_ORCAMENTARIO     object
REQUISICAO             int64
DIAGRAMA_DE_REDE       int64
TEXTO_DIAGRAMA        object
ANALISTA_FISCAL        int64
PLANEJADO            float64
COMPROMISSO          float64
TOTAL_MEDIDO         float64
MEDIDO_REAJUSTE      float64
SALDO                float64
STATUS                object
DATA_STATUS           object
DESVIO               float64
dtype: object

In [130]:
#inserindo dados no banco de dados
cnx = mysql.connector.connect(user='root',
                              password='root',
                              host='127.0.0.1',
                              database='bd_rpeotta')
cursor = cnx.cursor()

#cursor.execute('delete from bd_rpeotta.relatorio_emissao')

for i,df_colunas in df_copasa.iterrows():
    cursor.execute (
     '''
     insert into copasa(
     LOCALIDADE,PROJETO,PROG_ORCAMENTARIO,REQUISICAO,DIAGRAMA_DE_REDE,TEXTO_DIAGRAMA,ANALISTA_FISCAL,
     PLANEJADO,COMPROMISSO, TOTAL_MEDIDO,MEDIDO_REAJUSTE,SALDO,STATUS,DATA_STATUS,DESVIO) 
     
     values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
     ON DUPLICATE KEY UPDATE
         LOCALIDADE = VALUES(LOCALIDADE),         
         PROJETO = VALUES(PROJETO),
         PROG_ORCAMENTARIO = VALUES(PROG_ORCAMENTARIO),
         REQUISICAO = VALUES(REQUISICAO),
         DIAGRAMA_DE_REDE = VALUES(DIAGRAMA_DE_REDE),
         TEXTO_DIAGRAMA = VALUES(TEXTO_DIAGRAMA),
         ANALISTA_FISCAL = VALUES(ANALISTA_FISCAL),
         PLANEJADO = VALUES(PLANEJADO),
         COMPROMISSO = VALUES(COMPROMISSO),
         TOTAL_MEDIDO = VALUES(TOTAL_MEDIDO),
         MEDIDO_REAJUSTE = VALUES(MEDIDO_REAJUSTE),
         SALDO = VALUES(SALDO),
         STATUS = VALUES(STATUS),
         DATA_STATUS = VALUES(DATA_STATUS),
         DESVIO = VALUES(DESVIO);
    ''',(
         df_colunas['LOCALIDADE'],
         df_colunas['PROJETO'],
         df_colunas['PROG_ORCAMENTARIO'],
         df_colunas['REQUISICAO'],
         df_colunas['DIAGRAMA_DE_REDE'],
         df_colunas['TEXTO_DIAGRAMA'],
         df_colunas['ANALISTA_FISCAL'],
         df_colunas['PLANEJADO'],
         df_colunas['COMPROMISSO'],
         df_colunas['TOTAL_MEDIDO'],
         df_colunas['MEDIDO_REAJUSTE'],
         df_colunas['SALDO'],
         df_colunas['STATUS'],
         df_colunas['DATA_STATUS'],
         df_colunas['DESVIO']
         #df_colunas['COORDENADORES'],
         #df_colunas['PORCENTAGEM_MEDIDA'],
         #df_colunas['OBSERVACAO']
    )
)


cnx.commit()

cursor.close()
cnx.close()

#inserindo dados no banco de dados
cnx = mysql.connector.connect(user='root',
                              password='root',
                              host='127.0.0.1',
                              database='bd_rpeotta')
cursor = cnx.cursor()

#cursor.execute('delete from bd_rpeotta.relatorio_emissao')

for i,df_colunas in df_copasa.iterrows():
    cursor.execute (
     '''
     insert into copasa(
     LOCALIDADE,PROJETO,PROG_ORCAMENTARIO,REQUISICAO,DIAGRAMA_DE_REDE,TEXTO_DIAGRAMA,ANALISTA_FISCAL,
     PLANEJADO,COMPROMISSO, TOTAL_MEDIDO,MEDIDO_REAJUSTE,SALDO,STATUS,DATA_STATUS,DESVIO) 
     
     values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
     ON DUPLICATE KEY UPDATE
         LOCALIDADE = VALUES(LOCALIDADE),         
         PROJETO = VALUES(PROJETO),
         PROG_ORCAMENTARIO = VALUES(PROG_ORCAMENTARIO),
         REQUISICAO = VALUES(REQUISICAO),
         DIAGRAMA_DE_REDE = VALUES(DIAGRAMA_DE_REDE),
         TEXTO_DIAGRAMA = VALUES(TEXTO_DIAGRAMA),
         ANALISTA_FISCAL = VALUES(ANALISTA_FISCAL),
         PLANEJADO = VALUES(PLANEJADO),
         COMPROMISSO = VALUES(COMPROMISSO),
         TOTAL_MEDIDO = VALUES(TOTAL_MEDIDO),
         MEDIDO_REAJUSTE = VALUES(MEDIDO_REAJUSTE),
         SALDO = VALUES(SALDO),
         STATUS = VALUES(STATUS),
         DATA_STATUS = VALUES(DATA_STATUS),
         DESVIO = VALUES(DESVIO);
    ''',(
         df_colunas['LOCALIDADE'],
         df_colunas['PROJETO'],
         df_colunas['PROG_ORCAMENTARIO'],
         df_colunas['REQUISICAO'],
         df_colunas['DIAGRAMA_DE_REDE'],
         df_colunas['TEXTO_DIAGRAMA'],
         df_colunas['ANALISTA_FISCAL'],
         df_colunas['PLANEJADO'],
         df_colunas['COMPROMISSO'],
         df_colunas['TOTAL_MEDIDO'],
         df_colunas['MEDIDO_REAJUSTE'],
         df_colunas['SALDO'],
         df_colunas['STATUS'],
         df_colunas['DATA_STATUS'],
         df_colunas['DESVIO']
         #df_colunas['COORDENADORES'],
         #df_colunas['PORCENTAGEM_MEDIDA'],
         #df_colunas['OBSERVACAO']
    )
)


cnx.commit()

cursor.close()
cnx.close()